# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"



In [2]:
export_distances = False
aggregate_LU = False

if (aggregate_LU):
    prefix = "aggregated_"
else:
    prefix = "non_aggregated_"

In [3]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide")
proj = mature_biomass.projection().getInfo()

age_agbd = ee.Image(f"{data_folder}/age_agbd")
secondary = age_agbd.select("age")

if (export_distances):
    mature_biomass_vector = mature_biomass.reduceToVectors(geometry = secondary.geometry(), scale=500, maxPixels=1e13)
    distances_image = mature_biomass_vector.distance(searchRadius = 500000).toInt().updateMask(secondary)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.image.toAsset(
        image=distances_image,
        assetId = "projects/amazon-forest-regrowth/assets/distance_to_mature_biomass",
        description="distance_to_mature_biomass",
        region=secondary.geometry(),
        crs=proj['crs'],
        crsTransform=proj['transform'],
        maxPixels=1e13
    )
    task.start()



In [4]:

# Function to buffer and calculate mean
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(2000)))
    return buffered


In [66]:
categorical = ee.Image(f"{data_folder}/categorical")
distance = ee.Image(f"{data_folder}/distance_to_mature_biomass")

one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
biome_mask = categorical.select('biome').eq(1).Or(categorical.select('biome').eq(4))
combined_mask = one_hectare_mask.And(biome_mask)

categorical = categorical.updateMask(combined_mask)

dist_cat = distance.addBands(combined_mask)

dist_cat_sampled = dist_cat.stratifiedSample(
    numPoints = 100000, classBand = 'one_hectare_mask', region = age_agbd.geometry(), geometries = True
)

dist_cat_sampled_buffered = dist_cat_sampled.map(buffer_distance)

nearest_mature = mature_biomass.reduceRegions(dist_cat_sampled_buffered, ee.Reducer.firstNonNull(), 
                                        crs = proj['crs'], crsTransform = proj['transform']
)

# Export the mean_biomass_image as an asset
task = ee.batch.Export.table.toAsset(
    collection=nearest_mature,
    assetId = "projects/amazon-forest-regrowth/assets/nearest_mature",
    description="nearest_mature"
)

task.start()

In [5]:
nearest_mature = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/nearest_mature")

# # Convert the FeatureCollection to an image
nearest_mature_image = nearest_mature.reduceToImage(
    properties=['first'],  # Assuming 'first' is the current property name
    reducer=ee.Reducer.first()
).rename('nearest_mature')

export_task = ee.batch.Export.image.toAsset(
    image=nearest_mature_image,
    assetId = "projects/amazon-forest-regrowth/assets/nearest_mature_image",
    description='nearest_mature_export',
    crs=proj['crs'],
    crsTransform=proj['transform'],
    region = age_agbd.geometry(),  # Use the same region as in your sampling
    maxPixels=1e13  # Adjust if needed
)

# Start the export task
export_task.start()

In [35]:

def export_csv(year):
    if (year == 0):
        land_use_years = ee.Image(f"{data_folder}/land_use")
        name = f"{prefix}_all"
    else:
        land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
        name = f"{prefix}_{year}yr"
    
    nearest_mature = ee.Image(f"{data_folder}/nearest_mature")
    yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
    sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")

    combined_mask = land_use_years.select("last_LU").And(nearest_mature)
    unified_data = (
        age_agbd.addBands(
            [distance, nearest_mature, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
        ]).updateMask(combined_mask)
    )

    unified_data_sampled = unified_data.stratifiedSample(
        numPoints = 10000, classBand = 'biome', region = age_agbd.geometry(), geometries = True
    )

    task = ee.batch.Export.table.toDrive(
        collection=unified_data_sampled, description=name
    )
    task.start()

# task = ee.batch.Export.table.toAsset(
#     collection=pts_bm, description=name, assetId=f"projects/amazon-forest-regrowth/assets/{name}"
# )
# task.start()

export_csv(0)
export_csv(5)
export_csv(10)
export_csv(15)

In [8]:
gaul = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0')
br_shp = ee.FeatureCollection('projects/amazon-forest-regrowth/assets/raw/brasil_shp')

# Convert the FeatureCollection to an image
br_img = br_shp.reduceToImage(
    properties=['AREA_KM2'],  # Assuming 'first' is the current property name
    reducer=ee.Reducer.first()
).rename('br_img')

br_img = br_img.reproject(proj['crs'], proj['transform'])
br_img = br_img.gt(0)


south_american_countries = [
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
    'French Guiana', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

south_america = gaul.filter(ee.Filter.inList('ADM0_NAME', south_american_countries))

south_america_merged = south_america.union().geometry()


brazil_buffer = ee.Feature(br_shp.first()).buffer(-1000, maxError=5)

# edge_mask = brazil_buffer.difference(south_america_merged.geometry())
# secondary_notedge = secondary.updateMask(secondary.mask(brazil_buffer))

# map = geemap.Map()
# map.addLayer(south_america_merged, {}, "tst")
# map

# Create a mask by subtracting the buffered area from the South America feature

# # Export the mean_biomass_image as an asset
# task = ee.batch.Export.table.toAsset(
#     collection=brazil_buffer,
#     assetId = "projects/amazon-forest-regrowth/assets/brazil_buffer",
#     description="brazil_buffer"
# )
# task.start()

In [11]:
# Define a Polygon object.
polygon = ee.Geometry.Polygon([[
    [-122.092, 37.424],
    [-122.086, 37.418],
    [-122.079, 37.425],
    [-122.085, 37.423],
]])

# Define other inputs.
input_geom = ee.Geometry.BBox(-122.085, 37.415, -122.075, 37.425)

# Apply the symmetricDifference method to the Polygon object.
polygon_symmetric_difference = polygon.symmetricDifference(
    right=input_geom, maxError=1
)

# Print the result.
display('polygon.symmetricDifference(...) =', polygon_symmetric_difference)

# Display relevant geometries on the map.
m = geemap.Map()
m.set_center(-122.085, 37.422, 15)
m.add_layer(polygon, {'color': 'black'}, 'Geometry [black]: polygon')
m.add_layer(input_geom, {'color': 'blue'}, 'Parameter [blue]: input_geom')
m.add_layer(
    polygon_symmetric_difference,
    {'color': 'red'},
    'Result [red]: polygon.symmetricDifference',
)
m

'polygon.symmetricDifference(...) ='

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.symmetricDifference",
    "arguments": {
      "left": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    -122.092,
                    37.424
                  ],
                  [
                    -122.086,
                    37.418
                  ],
                  [
                    -122.079,
                    37.425
                  ],
                  [
                    -122.085,
                    37.423
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      },
      "maxError": {
        "functionInvocationValue": {
          "functionName": "ErrorMargin",
          "arguments": {
            "value": {
              "constantValue": 1
            }
          }
        }
      },
      "right": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    -122.08499999999998,
                    37.425
                  ],
                  [
                    -122.08499999999998,
                    37.415
                  ],
                  [
                    -122.07499999999999,
                    37.415
                  ],
                  [
                    -122.07499999999999,
                    37.425
                  ]
                ]
              ]
            },
            "geodesic": {
              "constantValue": false
            }
          }
        }
      }
    }
  }
})

Map(center=[37.422, -122.085], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [10]:
map = geemap.Map()
map.addLayer(br_img, {}, "br_img")
map.addLayer(brazil_buffer, {}, "brazil_buffer")
# map.addLayer(south_america_merged, {}, "largest_polygon")
map

KeyboardInterrupt: 

In [57]:

# Convert the MultiPolygon to a FeatureCollection of individual polygons
geometry_fc = ee.FeatureCollection(south_america_merged.geometries().map(lambda geom: ee.Feature(ee.Geometry(geom))))

# Calculate the area of each polygon and add it as a property
area_fc = geometry_fc.map(lambda f: f.set('area', f.geometry().area()))

# Sort the polygons by area in descending order
sorted_fc = area_fc.sort('area', False)

# Get the largest polygon (the first feature in the sorted collection)
largest_polygon = ee.Feature(sorted_fc.first()).geometry()


In [65]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
# mature_biomass_500m_countrywide = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide")
tst = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/nearest_mature")


map = geemap.Map()
map.addLayer(tst, {}, 'tst')
map.addLayer(mature_biomass, {"min": 0, "max": 400, "palette": ["blue", "red"]}, 'mature_biomass')
# map.addLayer(mature_biomass_100m, {"min": 0, "max": 400, "palette": ["blue", "red"]}, 'mature_biomass_100m')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…